In [8]:
import os
from PIL import Image
import pyocr
import numpy as np
import pyocr.builders
import matplotlib.pyplot as plt
import math
import difflib
import json
import time

In [2]:
path_tesseract = "C:\\Program Files\\Tesseract-OCR"
if path_tesseract not in os.environ["PATH"].split(os.pathsep):
    os.environ["PATH"] += os.pathsep + path_tesseract

tools = pyocr.get_available_tools()
tool = tools[0]

In [3]:
targets = ["normal", "wide", "fastlate"]
border = 215

In [22]:
def title(target):
    img = Image.open("../targets/" + target + ".png")
    
    data = {"6":"","11":""}
    
    rgb_img = img.convert('RGB')
    size = rgb_img.size
    
    crop_img = rgb_img.crop([0, 0, size[0]/2, size[1]/7])
    crop_size = crop_img.size
    
    img2 = Image.new('RGBA',crop_size)
    
    for x in range(crop_size[0]):
        for y in range(crop_size[1]):
            r,g,b = crop_img.getpixel((x,y))

            if r >= border and g >= border and b >= border:
                a = 255
            else:
                a = 0

            img2.putpixel((x,y),(a,a,a,255))
    
    crop_range = img2.convert('RGB').getbbox()
    # print(crop_range[1])
    img3 = img2.crop([crop_range[0],crop_range[1]-5,crop_range[2],(crop_range[3]) / 2])
    
    builder6 = pyocr.builders.TextBuilder(tesseract_layout=6)
    builder11 = pyocr.builders.TextBuilder(tesseract_layout=11)
    data["6"] = tool.image_to_string(img3, lang="jpn", builder=builder6)
    data["11"] = tool.image_to_string(img3, lang="jpn", builder=builder11)
    
    return data

In [27]:
def check (target):
    with open('../targets/tmp.json', encoding="utf-8") as f1:
        music = json.load(f1)

        data = {}

        for j in music:
            result = difflib.SequenceMatcher(None, target, j["title"]).ratio()

            if result > 0.6:
                data = ({"title":j["title"], "result":result})

        return data

In [29]:
start = time.perf_counter()
for target in targets:
    data = title(target)
    print(data)
    print(check(data["6"]))
    print(check(data["11"]))
    print(target + ": " + str(time.perf_counter() - start) + "\n")
print("avg: " + str((time.perf_counter() - start) / 3))

{'6': '', '11': 'プリキノダンス'}
{}
{'title': 'ブリキノダンス', 'result': 0.8571428571428571}
normal: 1.0395023000019137

{'6': 'Brand New Day', '11': 'Brand New Day'}
{'title': 'Brand New Day', 'result': 1.0}
{'title': 'Brand New Day', 'result': 1.0}
wide: 1.9573239000164904

{'6': '軸 てT  ほくらの16bit戦争', '11': '9)\n\nと\n\nほくらの16bit戦争'}
{'title': 'ぼくらの16bit戦争', 'result': 0.7142857142857143}
{'title': 'ぼくらの16bit戦争', 'result': 0.6896551724137931}
fastlate: 2.7936568000004627

avg: 0.9312537666701246
